# Question #1

In this question we are given vector b=[1,1,...,1] and matrix A(n*n) a hilbert matrix where n=5,10,20,100.

In [1]:
import numpy as np
from numpy import linalg as LA
from scipy.linalg import hilbert 

Hilbert matrix does not converge cause of it's norm. As shown below the norm of the matrix is greater than 1 therefore given any vector X(0) it does not necessarily converge.

In [2]:
for i in [5,10,20,100]:
    print(LA.norm(hilbert(i), 2)) #2-norm (largest sing. value)

1.567050691098231
1.7519196702651776
1.907134720407253
2.182696097757424


In the following three methods are used to solve AX=b.

# jacobi

In [3]:
def jacobi(A):
    b = np.ones_like(A[0])
    x = np.zeros_like(b)
    norm = 1
    count = 0
    
    while norm > 1.0e-4:
        count += 1
        x_new = np.zeros_like(x)
        
        for i in range(A.shape[0]):
            L = np.dot(A[i, :i], x[:i])
            U = np.dot(A[i, i+1:], x[i+1:])
            x_new[i] = (b[i] - L - U) / A[i, i]
            
        norm = LA.norm((x-x_new))
        if norm > 1.0e30:
            print("Norm is not converging")
            break
        x = x_new

    r = np.dot(A, x) - b
    print("Solution: " + "\n" + "{0}".format(x) + "\n" + "iteration: " + str(count))
    print("error: " + "\n" + "{0}".format(r))
    print("||X - X_new||: " + str(norm))

In [4]:
for i in [5,10,20,100]:
    A = hilbert(i)
    jacobi(A)
    print("-"*70)

Norm is not converging
Solution: 
[-4.19329581e+28 -8.75752401e+28 -1.14542740e+29 -1.32850183e+29
 -1.46209956e+29]
iteration: 55
error: 
[-1.86356028e+29 -1.29732273e+29 -1.01808844e+29 -8.43435862e+28
 -7.21975370e+28]
||X - X_new||: 1.1027774804118538e+30
----------------------------------------------------------------------
Norm is not converging
Solution: 
[-7.87682939e+27 -1.84297847e+28 -2.57982498e+28 -3.13909457e+28
 -3.58288771e+28 -3.94563363e+28 -4.24865752e+28 -4.50611490e+28
 -4.72787059e+28 -4.92105580e+28]
iteration: 33
error: 
[-6.91571059e+28 -5.39367007e+28 -4.53007727e+28 -3.93723857e+28
 -3.49523242e+28 -3.14926672e+28 -2.86941751e+28 -2.63752372e+28
 -2.44175469e+28 -2.27399790e+28]
||X - X_new||: 1.0157955747792407e+30
----------------------------------------------------------------------
Norm is not converging
Solution: 
[-2.18127651e+28 -5.55133093e+28 -8.20551954e+28 -1.04035086e+29
 -1.22737497e+29 -1.38943074e+29 -1.53175234e+29 -1.65806723e+29
 -1.77114245

# gauss-seidel

In [5]:
def gausseidel(A):
    b = np.ones_like(A[0])
    x = np.zeros_like(b)
    norm = 1
    count = 0

    while norm > 1.0e-4:
        count += 1
        x_new = np.zeros_like(x)
        
        for i in range(A.shape[0]):
            L = np.dot(A[i, :i], x_new[:i])
            U = np.dot(A[i, i+1:], x[i+1:])
            x_new[i] = (b[i] - L - U) / A[i, i]
            
        norm = LA.norm((x_new-x))
        if norm > 1.0e30:
            print("Norm is not converging")
            break
        if count > 200000:
            break
            
        x = x_new

    r = np.dot(A, x) - b
    print("Solution: " + "\n" + "{0}".format(x) + "\n" + "iteration: " + str(count))
    print("error: " + "\n" + "{0}".format(r))
    print("||X - X_new||: " + str(norm))

In [6]:
for i in [5,10,20,100]:
    A = hilbert(i)
    gausseidel(A)
    print("-"*70)

Solution: 
[    4.98455756  -119.7165259    628.79230052 -1118.19188446
   629.12121175]
iteration: 151927
error: 
[-6.53659754e-07  3.67287404e-06 -7.44219706e-06  4.64995081e-06
  0.00000000e+00]
||X - X_new||: 9.999950416456015e-05
----------------------------------------------------------------------
Solution: 
[   -2.86028273    34.02531709   113.2524923  -1119.59412607
  1381.76953268  1055.36104714  -674.02910512 -1720.3220299
  -953.45861959  1940.17861634]
iteration: 200001
error: 
[-2.14289577e-05  1.96751312e-04 -6.73184298e-04  7.21926566e-04
  2.17930026e-04 -3.75319779e-04 -3.65689803e-04  1.93267740e-05
  2.88331739e-04 -7.10542736e-15]
||X - X_new||: 0.01513552391401315
----------------------------------------------------------------------
Solution: 
[-8.04025169e-01  7.53696753e+01 -7.78717895e+02  2.13528105e+03
 -7.83783790e+02 -1.78486363e+03 -9.01084657e+02  4.47901107e+02
  1.37159548e+03  1.59698005e+03  1.21293698e+03  4.55961846e+02
 -4.13969788e+02 -1.17144565

# conjugate gradient

In [7]:
def conjgrad(A):
    b = np.ones_like(A[0])
    x = np.zeros_like(b)
    norm = 1
    count = 0

    r = b - np.dot(A, x)
    p = r
    r_old = np.dot(np.transpose(r), r)
    
    while norm > 1.0e-4:
        count += 1
        Ap = np.dot(A, p)
        alpha = r_old / np.dot(np.transpose(p), Ap)
        x_new = x + np.dot(alpha, p)
        r = r - np.dot(alpha, Ap)
        r_new = np.dot(np.transpose(r), r)
        p = r + (r_new/r_old)*p
        
        norm = LA.norm((x_new-x))
        if norm > 1.0e30:
            print("Norm is not converging")
            break
        if count > 200000:
            break
                
        r_old = r_new
        x = x_new
        
    error = np.dot(A, x) - b
    print("Solution: " + "\n" + "{0}".format(x) + "\n" + "iteration: " + str(count))
    print("error: " + "\n" + "{0}".format(error))
    print("||X - X_new||: " + str(norm))

In [8]:
for i in [5,10,20,100]:
    A = hilbert(i)
    conjgrad(A)
    print("-"*70)

Solution: 
[    5.  -120.   630. -1120.   630.]
iteration: 7
error: 
[-2.84217094e-14  2.13162821e-13  7.10542736e-14 -4.26325641e-14
 -9.94759830e-14]
||X - X_new||: 3.484327251238585e-07
----------------------------------------------------------------------
Solution: 
[-9.99660644e+00  9.89793611e+02 -2.37565025e+04  2.40212429e+05
 -1.26113900e+06  3.78346161e+06 -6.72620700e+06  7.00078518e+06
 -3.93795809e+06  9.23721581e+05]
iteration: 43
error: 
[ 1.19209290e-07 -5.20376489e-08 -1.05937943e-08 -1.42666977e-07
 -1.02561899e-07 -6.39120117e-08 -4.60422598e-08 -4.40049917e-08
 -6.09725248e-08 -1.03405910e-07]
||X - X_new||: 1.7867237652571182e-06
----------------------------------------------------------------------
Solution: 
[-1.09754395e+01  1.05097128e+03 -2.39568511e+04  2.20428291e+05
 -9.65350821e+05  1.99010298e+06 -1.25269795e+06 -1.34346779e+06
  8.83231061e+05  1.68795448e+06  3.88209199e+05 -1.30552015e+06
 -1.71054295e+06 -5.28240860e+05  1.20868159e+06  2.00288780e+06

The results show us for the given A and b in jacobi method we do not converge. As for gauss-seidel it seems to converge to a soluthion but it's taking too much time comparing to conjugate method in which it does converge to a solution with the error rate 1e-4.